In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.externals import joblib
import re

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
df = pd.read_csv("train.csv")
df = df.dropna()
data = df["title"]
data = data.apply(lambda x: x.lower())
data.head()

0    house dem aide: we didn’t even see comey’s let...
1    flynn: hillary clinton, big woman on campus - ...
2                    why the truth might get you fired
3    15 civilians killed in single us airstrike hav...
4    iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [4]:
batch_size = 5000
tokenizer = Tokenizer(nb_words=batch_size, split=' ')

tokenizer.fit_on_texts(data.values)
X = tokenizer.texts_to_sequences(data.values)

In [5]:
maxx = 0
c=0
for ll in X:
    c=len(ll)
    if c>maxx:
        maxx=c
print('Max',maxx)
X = pad_sequences(X)
print(len(X[0]))

Max 59
59


In [6]:
embed_dim = 128
lstm_out = 256

In [9]:
model = Sequential()
model.add(Embedding(batch_size, embed_dim,input_length = X.shape[1]))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 59, 128)           640000    
_________________________________________________________________
lstm (LSTM)                  (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 1,034,754
Trainable params: 1,034,754
Non-trainable params: 0
_________________________________________________________________


In [10]:
Y = pd.get_dummies(df['label']).values
Y

array([[0, 1],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(14628, 59) (14628, 2)
(3657, 59) (3657, 2)


In [13]:
model.fit(X_train, Y_train, epochs = 2, batch_size=64, verbose = 2)

Epoch 1/2
229/229 - 59s - loss: 0.2627 - accuracy: 0.8913
Epoch 2/2
229/229 - 55s - loss: 0.0961 - accuracy: 0.9645


In [14]:
def predict_news(inp):
    input_val = inp
    z = tokenizer.texts_to_sequences([input_val])
    z = pad_sequences(z, maxlen=59)
    z = np.array(z[0]).reshape(1, 59)
    pred = model.predict(z)
    if pred[0][0] > 0.5:
        result="true"
    else:
        result="fake"
    return result

In [15]:
predict_news("Pope has a new baby")

'fake'

In [16]:
predict_news("Pope is praying")

'true'